In [ ]:
pip install otter-grader


In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Exam2_Gradescope.ipynb")

## Exam 2

**Northeastern University Honor Code**\
On my honor, I pledge to uphold the values of honesty, integrity, and respect that are expected of me as a Northeastern student.

**Signature: (Please sign here)**

#### Import the Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf

#### 1. Create a DataFrame named **fin_data** by reading in only the columns labeled 'DATE', 'TICKER', 'PRC', 'CFACPR' and 'SHROUT' from the file entitled exam2_data.csv. Set the column labeled 'DATE' as the index. In addition, create two new columns: a) 'ADJ_PRC' to store the adjusted closing price, defined as PRC/CFACPR, and b) 'MKTCAP' to store the market capitalization, defined as PRC x SHROUT. Subsequently, permanently remove PRC and CFACPR from the DataFrame.
The file "exam2_data.csv" contains the following columns:
   * DATE
   * TICKER
   * PRC - The closing price
   * SHROUT - The number of shares outstanding
   * CFACPR - Cumulative factor to adjust the closing price
   * SPRTRN - The daily return on the S&P500 index

_Points:_ 10

In [ ]:
# Read the CSV file and select the required columns
fin_data = pd.read_csv("exam2_data.csv", usecols=["DATE", "TICKER", "PRC", "CFACPR", "SHROUT"])

# Set the 'DATE' column as the index
fin_data.set_index("DATE", inplace=True)

# Create the new columns 'ADJ_PRC' and 'MKTCAP'
fin_data["ADJ_PRC"] = fin_data["PRC"] / fin_data["CFACPR"]
fin_data["MKTCAP"] = fin_data["PRC"] * fin_data["SHROUT"]

# Remove the 'PRC' and 'CFACPR' columns
fin_data.drop(columns=["PRC", "CFACPR"], inplace=True)

# Display the DataFrame
print(fin_data)

---

#### Use your NEU ID # in the code below. There will be a significant point discount for failing to do so.

In [ ]:
params = {'NEU_ID': 100 
          #Replace 100 with your actual NEU ID#
         }

In [ ]:
np.random.seed(params['NEU_ID'])
all_tickers = ['AAPL', 'AXP', 'BA', 'BAC', 'C', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'T', 'UNH', 'V', 'VZ', 'WMT', 'XOM']

stock_list = sorted(np.random.choice(all_tickers,5,replace=False))

print(f'These are the tickers of the five stocks assigned to you: {" ".join(stock_list)}. They are stored in a \
variable called stock_list.')

### PART 1

#### 2. Use the DataFrame fin_data to create another DataFrame named mktcap to store the daily market capitalization for the stocks you were assigned. Each stock should have its own column in the DataFrame, labeled as the stock's ticker.

_Points:_ 9

In [ ]:
#Filter fin_data for the stocks in stock_list

filtered_fin_data = fin_data[fin_data['TICKER'].isin(stock_list)]

#Pivot the filtered data to create a DataFrame with stock tickers as columns and market capitalization values

mktcap = filtered_fin_data.pivot_table(values='MKTCAP', columns='TICKER', index=filtered_fin_data.index)

#Display the mktcap DataFrame

print(mktcap)

---

#### 3. Use the DataFrame mktcap to create another DataFrame named **avg_mktcap** that contains each stock's average daily market capitalization in the month of May of each year. The DataFrame should have 5 columns, one for each stock, and 5 rows, one for each year in the sample.

_Points:_ 7

In [ ]:
# Convert the index of mktcap DataFrame to datetime format
mktcap.index = pd.to_datetime(mktcap.index)

# Filter the data for the month of May
may_data = mktcap[mktcap.index.month == 5]

# Group the data by year and calculate the mean market capitalization for each stock
avg_mktcap = may_data.groupby(may_data.index.year).mean()

# Display the avg_mktcap DataFrame
print(avg_mktcap)

---

#### 4. Use the DataFrame avg_mktcap to find the stock with largest average daily market capitalization in the month of May in 2021? What was the magnitude? Store the two values in two variables labeled **stock_ans4** and **cap_ans4**.

_Points:_ 7

In [ ]:
# Find the stock with the largest average daily market capitalization in May 2021
stock_ans4 = avg_mktcap.loc[2021].idxmax()
cap_ans4 = avg_mktcap.loc[2021].max()

print(f"The stock with the largest average daily market capitalization in May 2021 is {stock_ans4} with a magnitude of {cap_ans4}.")


---

### PART 2

#### 5. Use the DataFrame fin_data to create another DataFrame named returns to store the simple daily return for the stocks you were assigned. Each stock should have its own column in the DataFrame, labeled as the stock's ticker.

_Points:_ 9

In [ ]:
# Filter fin_data for the stocks in stock_list
filtered_fin_data = fin_data[fin_data['TICKER'].isin(stock_list)]

# Pivot the filtered data to create a DataFrame with stock tickers as columns and adjusted closing prices
adj_price_data = filtered_fin_data.pivot_table(values='ADJ_PRC', columns='TICKER', index=filtered_fin_data.index)

# Calculate the simple daily return for each stock
returns = adj_price_data.pct_change()

# Display the returns DataFrame
print(returns)


---

#### 6. What is number of days the third stock you were assigned (third ticker in stock_list) had a negative return during the last three years in the sample period? Store the answer in a variable called **ans6**.

_Points:_ 7

In [ ]:
# Get the third stock's ticker from stock_list
returns.index = pd.to_datetime(returns.index)

third_stock = stock_list[2]

# Filter the returns DataFrame for the last three years in the sample period
last_three_years = returns[returns.index.year >= (returns.index.year.max() - 2)]

# Count the number of days with a negative return for the third stock
ans6 = (last_three_years[third_stock] < 0).sum()

print(f"The third stock ({third_stock}) had a negative return on {ans6} days during the last three years in the sample period.")


---

#### 7. Create another DataFrame named **returns_mon** to store the monthly simple returns for each stock in the sample. The DataFrame should have five columns, one for each stock in the sample.

_Points:_ 7

In [ ]:
returns_mon = returns.resample('M').apply(lambda x: (x + 1).prod() - 1)

---

#### 8. Create a new Pandas DataFrame named **ans8** to store the average monthly return for each stock during the second quarter of 2022. The shape of the DataFrame should be either (5,1) or (1,5).

_Points:_ 7

In [ ]:
returns_q2_2022 = returns_mon[(returns_mon.index.year == 2022) & (returns_mon.index.month >= 4) & (returns_mon.index.month <= 6)]
ans8 = returns_q2_2022.mean().to_frame().T

---

#### 9. Which stock had the highest total return over the sample period January 2018 - February 2020? What was the return? Store the two values in two variables labeled **stock_ans9** and **ret_ans9**.

_Points:_ 7

In [ ]:
# Filter the returns DataFrame for the sample period January 2018 - February 2020
sample_period = returns[(returns.index.year >= 2018) & (returns.index.year <= 2020) & ~((returns.index.year == 2020) & (returns.index.month > 2))]

# Calculate the total return for each stock during that time
total_returns = (sample_period + 1).cumprod().iloc[-1] - 1

# Find the stock with the highest total return and its magnitude
stock_ans9 = total_returns.idxmax()
ret_ans9 = total_returns.max()

print(f"The stock with the highest total return during January 2018 - February 2020 is {stock_ans9} with a return of {ret_ans9}.")

---

### PART 3

#### 10. Create a DataFrame named **sp_data** by reading in the columns labeled 'DATE' and 'SPRTRN' from the file entitled exam2_data.csv. The column labeled 'SPRTRN' contains the daily return for the S&P500 index. Set the column labeled 'DATE' as the index. Make sure the DataFrame does not have any duplicate observations. 

_Points:_ 9

In [ ]:
sp_data = pd.read_csv("exam2_data.csv", usecols=["DATE", "SPRTRN"], index_col="DATE", parse_dates=True)
sp_data = sp_data[~sp_data.index.duplicated(keep='first')]

---

#### 11. Create a DataFrame called djia_data to store the simple daily return for the Dow Jones Index, computed using the Adjusted Closing Price from Yahoo!Finance. The sample period is January 1st, 2018 to December 31st, 2022. The ticker for the index is ^DJI. The DataFrame should have an index named 'DATE', one column named 'DJIA_RET' and no missing values.

_Points:_ 9

In [ ]:
import yfinance as yf

djia = yf.download("^DJI", start="2018-01-01", end="2022-12-31")
djia_data = djia['Adj Close'].pct_change().dropna().to_frame()
djia_data.columns = ['DJIA_RET']
djia_data.index.name = 'DATE'

---

#### 12. Create a new DataFrame named index_data by inner merging on 'DATE' the DataFrame sp_data with the DataFrame djia_data.

_Points:_ 5

In [ ]:
index_data = sp_data.merge(djia_data, left_index=True, right_index=True)

---

#### 13. Create a Pandas DataFrame named **ans13** to store the annual correlation coefficients between the daily returns on the S&P500 and Dow Jones Index. The shape of the DataFrame should be either (5,1) or (1,5).

_Points:_ 9

In [ ]:
def compute_corr(x):
    return x['SPRTRN'].corr(x['DJIA_RET'])

annual_corr = index_data.resample('Y').apply(compute_corr).to_frame().T
annual_corr.columns = annual_corr.columns.year

# Display the ans13 DataFrame
print(annual_corr)


---

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)